In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random as random
import matplotlib.animation as animation
import pandas as pd

In [1]:
def insert(digit, frame, position):
    max_value = np.max(digit)
    for i, line in enumerate(digit):
        for j, pixel in enumerate(line):
            frame_value = frame[position[0]+i, position[1]+j]
            if frame_value + pixel > max_value:
                frame[position[0]+i, position[1]+j] = max_value
                #saturation
            else:
                frame[position[0]+i, position[1]+j] += pixel
    return frame

In [3]:
def create_frame(frame_size, digits, positions):
    frame = np.zeros((frame_size, frame_size))
    for k, digit in enumerate(digits):
        position = positions[k]
        frame = insert(digit, frame, position)
    return frame

In [4]:
def mirror_direction(direction, position, frame_size):
    #only called if digit hits a wall, so either x or y equals 0 or frame_size-1
    dx, dy = direction
    x,y = position
    if (x==0 or x==frame_size-1) and y!=0 and y!=frame_size-1:
        dx = -dx
    if (y==0 or y==frame_size-1) and x!=0 and x!=frame_size-1:
        dy = -dy
    if (x==0 or x==frame_size-1) and (y==0 or y==frame_size-1):
        dx, dy = -dx, -dy
    return [dx, dy]

In [5]:
def closest_frontier(position, frame_size, digit_size):
    x,y = position
    neighbor_frontiers = [(frame_size-1,y), (0,y), (x, frame_size-1), (x,0)]
    distances = []
    distances.append(frame_size - (x+digit_size))
    distances.append(x)
    distances.append(frame_size - (y+digit_size))
    distances.append(y)
    return neighbor_frontiers[distances.index(min(distances))]

In [6]:
def non_null_sample(max_value, number):
    values = list(np.arange(-max_value, max_value+1))
    sample = []
    for i in range(number-1):
        sample.append(random.choice(values))
    #for the last value, we cannot add a zero if there are only zero in the selected values
    if list(set(sample)) != [0]:
        sample.append(random.choice(values))
    else:
        values.remove(0)
        sample.append(random.choice(values))
    return sample

In [7]:
def new_position(position, direction, speed):
    x,y = position
    dx, dy = direction
    return [x+speed*dx, y+speed*dy]

In [8]:
def out_of_frame(position, frame_size, digit_size):
    for i, dimension in enumerate(position) :

        if dimension + digit_size  > frame_size:
            return True
        if dimension < 0:
            return True
    return False

In [1]:
def in_the_center(position, frame_size, speed, digit_size, max_jump):
    security_distance = abs(speed*max_jump)+ digit_size
    for dimension in position :
        if abs(frame_size-dimension)<security_distance or dimension < security_distance:
            return False
    return True

In [10]:
def insert_and_label(digit, digit_label, frame, proba, position):

    max_value = np.max(digit)
    
    for i, line in enumerate(digit):
        for j, pixel in enumerate(line):
            
            frame_value = frame[position[0]+i, position[1]+j]
            
            if frame_value + pixel > max_value:
                frame[position[0]+i, position[1]+j] = max_value
                #saturation
            else:
                frame[position[0]+i, position[1]+j] += pixel

            if pixel != 0:
                proba[position[0]+i][position[1]+j] = 1
            #where a non-zero digit pixel is inserted, a one is inserted in the same position of the corresponding matrix
            
    return frame, proba

In [12]:
def generate_dataset(frame_size, digits_set, digits_labels_set, n_digits, n_frames):
    #version with 10 frames instead of one frame of vectors
    
    digit_size = len(digits_set[0][0])
    
    frames = []
    labels = []
    
    for i in range(n_frames):
        frame = np.zeros((frame_size, frame_size))
        label = [np.zeros((frame_size, frame_size)) for i in range(len(list(set(digits_labels_set))))]
        #a 280x280 matrix for each digit

        indexes = random.sample(range(len(digits_set)), n_digits)
        digits = digits_set[indexes]
        digits_labels = digits_labels_set[indexes]
        
        positions = [random.sample(range(frame_size-digit_size-1),2) for i in range(n_digits)]

        
        for k, digit in enumerate(digits):
            digit_label = digits_labels[k]

            digit_proba = label[digit_label]
            frame, digit_proba = insert_and_label(digit, digit_label, frame, digit_proba, positions[k])
            label[digit_label] = digit_proba

        #print(frame)
        
    
        
        labels.append(label)
        frames.append(frame)

        
    return np.array(frames), np.array(labels)